In [1]:
import pandas as pd
import re

In [2]:
def vietnamese_numerical_to_numeric(vietnamese_str):
    numerical_dict = {
        'tỷ': 10**9,
        'triệu': 10**6,
        'nghìn': 10**3,
        'đồng': 1,
        'đ': 1
    }

    value = 0
    current_value = 0
    for word in vietnamese_str.split():
        if word in numerical_dict:
            current_value *= numerical_dict[word]
            value += current_value
            current_value = 0
        else:
            current_value += float(word.replace('tỷ', '').replace('triệu', '').replace('nghìn', ''))

    return value


In [3]:
df_house = pd.read_csv('../data1/house_today(2024-04-24).csv')
df_house_info = pd.read_csv('../data1/house_info_today(2024-04-24).csv')

In [4]:
df_house.head()

,name,district,price,bedroom,wc,acreage,link,date
0,"Bán Nhà Mới Đẹp,Hẻm Ba Gác sát HXH, Lê Quang Đ...","Quận Bình Thạnh, TPHCM",6 tỷ 950 triệu,8 PN,9 WC,100 m2,https://mogi.vn/quan-binh-thanh/mua-nha-hem-ng...,2024-04-24
1,Bán nhà Hẻm Ba Gác Thông Lê Quang Định P.7 Q.B...,"Quận Bình Thạnh, TPHCM",5 tỷ,4 PN,4 WC,63 m2,https://mogi.vn/quan-binh-thanh/mua-nha-hem-ng...,2024-04-24
2,"Bán Nhà Mới Đẹp , HXH đỗ cửa Thống Nhất P.11 G...","Quận Gò Vấp, TPHCM",6 tỷ 500 triệu,5 PN,6 WC,33 m2,https://mogi.vn/quan-go-vap/mua-nha-hem-ngo/ba...,2024-04-24
3,Bán nhà mặt tiền đường Số 66 Thảo Điền 4x25m v...,"Quận 2 (TP. Thủ Đức), TPHCM",13 tỷ,3 PN,5 WC,45 m2,https://mogi.vn/quan-2/mua-nha-mat-tien-pho/ba...,2024-04-24
4,Bán toà nhà VP Gold Star Lý Thường Kiệt. hầm 9...,"Quận 10, TPHCM",75 tỷ,8 PN,9 WC,120 m2,https://mogi.vn/quan-10/mua-nha-mat-tien-pho/b...,2024-04-24


In [5]:
df_house.shape

(5855, 8)

In [6]:
def preprocess_house(df_house):
    df_house = df_house[['link','price']]
    df_house['link'] = df_house['link'].apply(lambda x: re.search(r"\-id([0-9]*)",x).group(1))
    df_house.rename(columns={'price':'price(billionVND)', 'link':'id'},inplace=True)
    df_house['id'] = pd.to_numeric(df_house['id'])

    # Drop rows with NaN values in the 'id' column
    df_house.dropna(subset=['id'], inplace=True)

    df_house['id'] = df_house['id'].astype(int)

    df_house['price(billionVND)'] = df_house['price(billionVND)'].apply(lambda x: vietnamese_numerical_to_numeric(x)/1000000000)
    df_house.drop_duplicates(subset=['id'], inplace=True)
    return df_house

In [7]:
df_house = preprocess_house(df_house) 

C:\Users\anhtu\AppData\Local\Temp\ipykernel_18636\1686690951.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_house['link'] = df_house['link'].apply(lambda x: re.search(r"\-id([0-9]*)",x).group(1))
C:\Users\anhtu\AppData\Local\Temp\ipykernel_18636\1686690951.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_house.rename(columns={'price':'price(billionVND)', 'link':'id'},inplace=True)
C:\Users\anhtu\AppData\Local\Temp\ipykernel_18636\1686690951.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .l

In [8]:
df_house.shape

(5769, 2)

In [9]:
df_house_info.head()

,address,latitude,longitude,describe,area_used,area,bedroom,wc,juridical,date_submitted,id,seller,seniority,phone
0,"Lê Quang Định, Phường 5, Quận Bình Thạnh, TPHCM",10.812072,106.687519,Mô tả: +DT: 4.05*12.5 nở hậu tài Lộc 7.5m; DTC...,"109,5 m2","63 m2 (4,05x12)",4.0,4.0,Sổ hồng,24/04/2024,22521691,Châu Minh Tuấn,2 năm 5 tháng,908935028
1,"Lê Quang Định, Phường 7, Quận Bình Thạnh, TPHCM",10.807009,106.691612,"Mô tả:+DT: 3,2x9,5+ Kết cấu: 1 trệt, 1 lửng, 2...","126,4 m2","33 m2 (3,2x9,5)",5.0,6.0,Sổ hồng,24/04/2024,22524911,Châu Minh Tuấn,2 năm 5 tháng,908935028
2,"Thống Nhất, Phường 11, Quận Gò Vấp, TPHCM",10.839189,106.661079,"Mô tả:*DT: 5*9👉 Kết cấu: 5 tầng gồm trệt, lửng...",225 m2,45 m2 (5x9),3.0,5.0,Sổ hồng,24/04/2024,22539926,Châu Minh Tuấn,2 năm 5 tháng,908935028
3,"26 Đường số 66, Phường Thảo Điền, Quận 2 (TP. ...",10.812520,106.729592,"Bán nhà mặt tiền đường Số 62 Thảo Điền, Quận 2...",120 m2,120 m2 (4x25),8.0,9.0,Sổ hồng,24/04/2024,22539325,Cao Ngọc Nam,3 năm 3 tháng,902989755
4,"Lý Thường Kiệt, Phường 14, Quận 10, TPHCM",10.772981,106.661602,Bán toà nhà văn phòng Gold Star Lý Thường Kiệt...,2850 m2,300 m2 (20x16),20.0,20.0,Sổ hồng,24/04/2024,22492767,NGUYỄN DƯƠNG,5 năm 9 tháng,935665544


In [10]:
df_house_info.shape

(5853, 14)

In [11]:
def preprocess_houseinfo(df_house_info, df_house):
    df_house_info.drop_duplicates(subset=['id'], inplace=True)
    df_merge = df_house_info.merge(df_house,on='id')
    return df_merge

In [12]:
df_merge = preprocess_houseinfo(df_house_info, df_house)

In [13]:
missing_ratio = df_merge.isnull().sum()
missing_ratio = (missing_ratio / df_merge.shape[0])*100
missing_ratio

address               0.000000
latitude              1.838044
longitude             1.838044
describe              0.000000
area_used            42.118953
area                  0.000000
bedroom               1.456563
wc                    1.075082
juridical             0.000000
date_submitted        0.000000
id                    0.000000
seller                0.000000
seniority             0.000000
phone                 0.000000
price(billionVND)     0.000000
dtype: float64